# テキスト生成モデルの構築

## 準備

### Googleドライブのマウント

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Googleドライブ直下に「text_generation」フォルダーを作り、そのフォルダーで作業する

In [ ]:
import os
path = '/content/drive/MyDrive/text_generation/'
if not os.path.exists(path):
    os.makedirs(path)

In [ ]:
%cd /content/drive/MyDrive/text_generation/

/content/drive/MyDrive/text_generation


下記のコードで、割り当てられたGPUの使用可能容量を確認している。

In [ ]:
!pip install gputil
import psutil
import humanize
import os
import GPUtil as GPU
GPUs = GPU.getGPUs()
# XXX: only one GPU on Colab and isn’t guaranteed
gpu = GPUs[0]
def printm():
    process = psutil.Process(os.getpid())
    print("Gen RAM Free: " + humanize.naturalsize( psutil.virtual_memory().available ), " | Proc size: " + humanize.naturalsize( process.memory_info().rss))
    print("GPU RAM Free: {0:.0f}MB | Used: {1:.0f}MB | Util {2:3.0f}% | Total {3:.0f}MB".format(gpu.memoryFree, gpu.memoryUsed, gpu.memoryUtil*100, gpu.memoryTotal))
printm()

Gen RAM Free: 26.2 GB  | Proc size: 118.6 MB
GPU RAM Free: 15109MB | Used: 0MB | Util   0% | Total 15109MB


### モジュールのインポート

In [ ]:
!pip install -r requirements.txt

In [ ]:
from pathlib import Path
import re
import math
import time
import copy
from tqdm import tqdm
import pandas as pd
import tarfile
import neologdn
from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn
from torch import optim
from torch.utils.data import DataLoader
from transformers import T5ForConditionalGeneration, T5Tokenizer
import settings

In [ ]:
%load_ext autoreload
%autoreload 2
pd.set_option('max_rows', 1000)
pd.set_option('max_columns', 1000)
pd.set_option('max_colwidth', 300)

## データの取得

In [ ]:
!wget -O ldcc-20140209.tar.gz https://www.rondhuit.com/download/ldcc-20140209.tar.gz

--2021-09-27 08:24:45--  https://www.rondhuit.com/download/ldcc-20140209.tar.gz
Resolving www.rondhuit.com (www.rondhuit.com)... 59.106.19.174
Connecting to www.rondhuit.com (www.rondhuit.com)|59.106.19.174|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 8855190 (8.4M) [application/x-gzip]
Saving to: ‘ldcc-20140209.tar.gz’

ldcc-20140209.tar.g 100%[===================>]   8.44M  3.55MB/s    in 2.4s    

2021-09-27 08:24:48 (3.55 MB/s) - ‘ldcc-20140209.tar.gz’ saved [8855190/8855190]



In [ ]:
with tarfile.open("ldcc-20140209.tar.gz") as tar:
    tar.extractall()

In [ ]:
file_paths = []
for dir_path in Path('text').glob('*/**'):
    for file_path in dir_path.glob('*'):
        if dir_path.name in file_path.name:
            file_paths.append(file_path)

data = []
for file_path in file_paths:
    with open(file_path, 'r') as file:
        lines = file.readlines()[2:]
        title = lines[0]
        body = ''.join(lines[1:])
        data.append((title, body))
data = pd.DataFrame(data, columns=['title', 'body'])

## 学習データの作成

### データの前処理

In [ ]:
def preprocess_text(text):
    text = re.sub(r'[\r\t\n\u3000]', '', text)
    text = neologdn.normalize(text)
    text = text.lower()
    text = text.strip()
    return text

In [ ]:
data = data.assign(
    title=lambda x: x.title.map(lambda y: preprocess_text(y)),
    body=lambda x: x.body.map(lambda y: preprocess_text(y))
)

In [ ]:
data.head()

,title,body
0,【dvdエンター!】誘拐犯に育てられた女が目にした真実は、孤独か幸福か,2005年11月から翌2006年7月まで読売新聞にて連載された、直木賞作家・角田光代による初の長編サスペンス『八日目の蝉』。2010年に檀れいと北乃きいの出演によりテレビドラマ化された同作が、2011年4月に永作博美と井上真央の出演によって映画化。そして、劇場公開から半年が過ぎた10月28日、dvd&ブルーレイとなって発売されました。八日目の蝉妻子ある男と愛し合い、その子を身ごもりながら、あきらめざるをえなかった女。彼女は同時に、男の妻が子供を産んだことを知る。その赤ん坊を見に行った女は、突発的にその子を連れ去り、逃避行を続けた挙句、小豆島に落ち着き、母と娘として暮らしはじめる。不倫相...
1,藤原竜也、中学生とともにロケット打ち上げに成功,"「アンテナを張りながら生活をしていけばいい」2月28日、映画『おかえり、はやぶさ』(3月10日より公開)と文部科学省とのタイアップとして、千代田区立神田一橋中学校に通う中学三年生と“宇宙""をテーマにした特別授業を行った。本作で主演を務める藤原竜也がサプライズで登場し、イベントを盛り上げた。イベントの挨拶で奥村展三文部科学副大臣は「みなさんは大きな夢を持っているということで、実現するために、文部科学省も環境を作り応援していますので、チャレンジ精神で頑張ってください。」と参加した生徒たちにエールを送った。今回の特別授業は、2部制で行われた。第1部では、ロケットの中はどうなっているのか、発射..."
2,『戦火の馬』ロイヤル・プレミアにウィリアム王子&キャサリン妃が出席,3月2日より全国ロードショーとなる、スティーブン・スピルバーグの待望の監督最新作『戦火の馬』。早くもアカデミー賞最有力候補として大きな注目を集めている同作のロンドンロイヤル・プレミアが、現地時間8日(日本時間9日未明)に行われた。本プレミアは、英国王室ウィリアム王子とハリー王子が運営する慈善団体のチャリティイベントとして開催され、会場には昨年4月にロイヤル・ウェディングを挙げたウィリアム王子とキャサリン妃も出席。結婚後、初の映画プレミア公式出席に会場のレスタースクエアは、ファンの大歓声に包まれた。『戦火の馬』は、第一次大戦を舞台に、悲劇に見舞われながらも希望を信じて生き抜く人間たちの姿...
3,香里奈、女子高生100人のガチンコ質問に回答「ラーメンも食べる」,女優の香里奈が18日、都内で行われた映画『ガール』(5月26日公開)の女子高生限定試写会にサプライズで出席し、約100人の女子高生からのガチンコ質問に答えた。・映画『ガール』特集-ナノケアが当たるキャンペーン実施中作品の上映終了後、スペシャルゲストの登場がアナウンスされると、会場後方の扉から主演の香里奈が登場。サプライズの演出に「キャー!」「カワイイ!」「ヤバイ!」と悲鳴に近い歓声が上がった。香里奈の女子高生時代はアムラー香里奈は「向井(理)君じゃなくてごめんね」と自虐ネタで笑いを誘うと、現役の女子高生を目の前に「自分も(女子高生に)戻りたいなと思います。まだまだ同じだと思っていたんで...
4,ユージの前に立ちはだかったjoy「僕はakbの高橋みなみを守る」,"5日、東京・千代田区の内幸町ホールにて、映画『キャプテン・アメリカ/ザ・ファースト・アベンジャー』の公開を記念して、宿命のライバル対決イベントが行われた。先日行われた“スーパーソルジャー計画""イベントにて、上島竜兵から生まれ変わって誕生した和製キャプテン・アメリカのユージは、世界最初のヒーローとして、日本各地を飛び回りpr活動に獅子奮迅の活躍をしていた。<変身の時の様子はこちら>・上島竜兵が「出川には負けない!」と極秘実験でイケメンマッチョに変身今回のイベントもバイクに乗って、さっそうと登場。キャプテン・アメリカを応援する“キャプアメ・ガールズ""を引き連れての華やかなイベントになるはず..."


### データを学習用データと確認用データに分離し、ベクトル化

In [ ]:
def convert_batch_data(train_data, valid_data, tokenizer):

    def generate_batch(data):

        batch_src, batch_tgt = [], []
        for src, tgt in data:
            batch_src.append(src)
            batch_tgt.append(tgt)

        batch_src = tokenizer(
            batch_src, max_length=settings.max_length_src, truncation=True, padding="max_length", return_tensors="pt"
        )
        batch_tgt = tokenizer(
            batch_tgt, max_length=settings.max_length_target, truncation=True, padding="max_length", return_tensors="pt"
        )

        return batch_src, batch_tgt

    train_iter = DataLoader(train_data, batch_size=settings.batch_size_train, shuffle=True, collate_fn=generate_batch)
    valid_iter = DataLoader(valid_data, batch_size=settings.batch_size_valid, shuffle=True, collate_fn=generate_batch)

    return train_iter, valid_iter

In [ ]:
tokenizer = T5Tokenizer.from_pretrained(settings.MODEL_NAME, is_fast=True)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    data['title'], data['body'], test_size=0.2, random_state=42, shuffle=True
)

train_data = [(src, tgt) for src, tgt in zip(X_train, y_train)]
valid_data = [(src, tgt) for src, tgt in zip(X_test, y_test)]

train_iter, valid_iter = convert_batch_data(train_data, valid_data, tokenizer)

## ニュースタイトルから本文を生成するテキスト生成モデルの構築

### モデル定義クラス

In [ ]:
class T5FineTuner(nn.Module):
    
    def __init__(self):
        super().__init__()

        self.model = T5ForConditionalGeneration.from_pretrained(settings.MODEL_NAME)

    def forward(
        self, input_ids, attention_mask=None, decoder_input_ids=None,
        decoder_attention_mask=None, labels=None
    ):
        return self.model(
            input_ids,
            attention_mask=attention_mask,
            decoder_input_ids=decoder_input_ids,
            decoder_attention_mask=decoder_attention_mask,
            labels=labels
        )

### モデル学習処理関数

In [ ]:
def train(model, data, optimizer, PAD_IDX):
    
    model.train()
    
    loop = 1
    losses = 0
    pbar = tqdm(data)
    for src, tgt in pbar:
                
        optimizer.zero_grad()
        
        labels = tgt['input_ids'].to(settings.device)
        labels[labels[:, :] == PAD_IDX] = -100

        outputs = model(
            input_ids=src['input_ids'].to(settings.device),
            attention_mask=src['attention_mask'].to(settings.device),
            decoder_attention_mask=tgt['attention_mask'].to(settings.device),
            labels=labels
        )
        loss = outputs['loss']

        loss.backward()
        optimizer.step()
        losses += loss.item()
        
        pbar.set_postfix(loss=losses / loop)
        loop += 1
        
    return losses / len(data)

In [ ]:
def evaluate(model, data, PAD_IDX):
    
    model.eval()
    losses = 0
    with torch.no_grad():
        for src, tgt in data:

            labels = tgt['input_ids'].to(settings.device)
            labels[labels[:, :] == PAD_IDX] = -100

            outputs = model(
                input_ids=src['input_ids'].to(settings.device),
                attention_mask=src['attention_mask'].to(settings.device),
                decoder_attention_mask=tgt['attention_mask'].to(settings.device),
                labels=labels
            )
            loss = outputs['loss']
            losses += loss.item()
        
    return losses / len(data)

### モデルの学習

In [ ]:
model = T5FineTuner()
model = model.to(settings.device)

optimizer = optim.Adam(model.parameters())

PAD_IDX = tokenizer.pad_token_id
best_loss = float('Inf')
best_model = None
counter = 1

for loop in range(1, settings.epochs + 1):

    start_time = time.time()

    loss_train = train(model=model, data=train_iter, optimizer=optimizer, PAD_IDX=PAD_IDX)

    elapsed_time = time.time() - start_time

    loss_valid = evaluate(model=model, data=valid_iter, PAD_IDX=PAD_IDX)

    print('[{}/{}] train loss: {:.4f}, valid loss: {:.4f} [{}{:.0f}s] counter: {} {}'.format(
        loop, settings.epochs, loss_train, loss_valid,
        str(int(math.floor(elapsed_time / 60))) + 'm' if math.floor(elapsed_time / 60) > 0 else '',
        elapsed_time % 60,
        counter,
        '**' if best_loss > loss_valid else ''
    ))

    if best_loss > loss_valid:
        best_loss = loss_valid
        best_model = copy.deepcopy(model)
        counter = 1
    else:
        if counter > settings.patience:
            break

        counter += 1

100%|██████████| 737/737 [09:32<00:00,  1.29it/s, loss=4.05]


[1/1000] train loss: 4.0486, valid loss: 3.5943 33s] counter: 1 **


100%|██████████| 737/737 [09:32<00:00,  1.29it/s, loss=3.59]


[2/1000] train loss: 3.5864, valid loss: 3.4997 32s] counter: 1 **


100%|██████████| 737/737 [09:31<00:00,  1.29it/s, loss=3.33]


...

[23/1000] train loss: 1.1467, valid loss: 4.6096 35s] counter: 19 


100%|██████████| 737/737 [09:37<00:00,  1.28it/s, loss=1.09]


[24/1000] train loss: 1.0931, valid loss: 4.7337 37s] counter: 20 


100%|██████████| 737/737 [09:42<00:00,  1.27it/s, loss=1.04]


[25/1000] train loss: 1.0377, valid loss: 4.8071 42s] counter: 21 


## モデルの保存

In [ ]:
model_dir_path = Path('model')
if not model_dir_path.exists():
    model_dir_path.mkdir(parents=True)

In [ ]:
tokenizer.save_pretrained(model_dir_path)
model.model.save_pretrained(model_dir_path)

## 学習したモデルを使ってニュース本文の生成

In [ ]:
def generate_text_from_model(title, trained_model, tokenizer, num_return_sequences=1):

    trained_model.eval()
    
    title = preprocess_text(title)
    batch = tokenizer(
        [title], max_length=settings.max_length_src, truncation=True, padding="longest", return_tensors="pt"
    )

    # 生成処理を行う
    outputs = trained_model.generate(
        input_ids=batch['input_ids'].to(settings.device),
        attention_mask=batch['attention_mask'].to(settings.device),
        max_length=settings.max_length_target,
        repetition_penalty=8.0,   # 同じ文の繰り返し（モード崩壊）へのペナルティ
        # temperature=1.0,  # 生成にランダム性を入れる温度パラメータ
        # num_beams=10,  # ビームサーチの探索幅
        # diversity_penalty=1.0,  # 生成結果の多様性を生み出すためのペナルティパラメータ
        # num_beam_groups=10,  # ビームサーチのグループ
        num_return_sequences=num_return_sequences,  # 生成する文の数
    )

    generated_texts = [
        tokenizer.decode(ids, skip_special_tokens=True, clean_up_tokenization_spaces=False) for ids in outputs
    ]

    return generated_texts

In [ ]:
tokenizer = T5Tokenizer.from_pretrained(model_dir_path)
trained_model = T5ForConditionalGeneration.from_pretrained(model_dir_path)

In [ ]:
index = 0
title = valid_data[index][0]
body = valid_data[index][1]
generated_texts = generate_text_from_model(
    title=title, trained_model=trained_model, tokenizer=tokenizer, num_return_sequences=1
)
print('□ タイトル')
print(title)
print()
print('□ 生成本文')
print(generated_texts[0])
print()
print('□ 教師データ本文')
print(body)

□ タイトル
あなたの厄年どでした?

□ 生成本文
みなさんは、年末の大掃除やお正月の準備はしていますか?「大掃除」と聞いて、「今年の厄年は何回したっけ?」という疑問が脳裏をよぎることはありませんか?会社員の美枝子さん(仮名・36歳)も、その時期についてこんな質問をしたんだろうなぁと思ったそうです。「そろそろ梅雨明けまで自分の体調管理に気を配らなくちゃいけない時期に差し掛かりたい!」と思いつつも、忙しくて家に帰れない日々にイライラしたり、仕事で疲れてしまってなかなか乗り切れないのが悩みの種ですね。一年のうち、2月15日くらいは肌寒い日が続きましたね。私は1ヶ月に1回は会社帰りに同僚と3人で手を繋いで出かけたり、色々と手を繋いだりして過ごす人が多かったので、職場の女子たちはそんな厄年を気にしなくていいように、普段から念入りにチェックするように心がけながら過ごしているようです。ちなみに昨年の厄年は『二宮御成婚』という超富裕層向けのサービス業を営む芳子さん。さっそく結婚までの紆余曲折を経て結婚した美枝子さんの家族も、それぞれパターンにあった厄払い法を教えてもらいました。最初のうちは、嫁さんが亡くなった後のことはあまり覚えていなかったので、私も母も慌てて様子見だったのですが...。その後、親戚の家に集まり、厄払いを始めた

□ 教師データ本文
“女30代"=人生の節目、転機、結婚適齢期、そして「厄」に振り回されるお年頃といっても過言ではない。というのも、女の厄年は数え年(※1)で、19歳、33歳(大厄)、37歳(小厄)と30代で大厄・小厄と言われている厄が2回もあり、その厄には前厄と後厄がついてまわる。つまり、30代のうちの6年間は「厄」が事あるごとに脳内を駆け抜けてゆく。厄年の起源は諸説あり、すでに平安時代では、貴族たちの間で厄払いが行われていたらしい。現在の厄年が定着したのは江戸時代と言われていて、特に33歳、37歳というのは女性にとって昔も今も精神面や健康面に変化が生じやすい時期だとされているのだろう。厄年を気にするか気にしないかは人それぞれだが、今回は30代の厄年を終えた女性、只今厄年真っ最中の女性たちに厄年体験談をうかがった。“厄年気にする派"のナオミさん(39歳)は「大厄(32)の時は彼氏の浮気と借金が発覚して、ストレスで血尿と激しい胃痛の末、人生初の胃カメラ

In [ ]:
index = 12
title = valid_data[index][0]
body = valid_data[index][1]
generated_texts = generate_text_from_model(
    title=title, trained_model=trained_model, tokenizer=tokenizer, num_return_sequences=1
)
print('□ タイトル')
print(title)
print()
print('□ 生成本文')
print(generated_texts[0])
print()
print('□ 教師データ本文')
print(body)

□ タイトル
あなたが同性から嫌われる理由

□ 生成本文
女の敵は女。職場では上司や先輩との付き合い、結婚をすれば姑・小姑との付き合い、子供ができればママ友との付き合いに悩む女性たちは多い。同性に嫌われては平和な日常生活を過ごすことができないといっても過言ではない。嫌われないようにするにはどうすればいいのか。まずは周りの独女たちにどんな女性が嫌いかを聞いてみた。○性格がちょっとイチャメチャで、男性に対しては警戒心が強い。○普段は自分より何年も若く見られている気がするのだが、なぜかいつも自分が本気で自分を省みようとするところがある。○普段から目につく異性に対して態度ががらりと変わってぶりっこになる。○外見とは裏腹に他人を見下す天然な女性のことをいう。○容姿に裏表がある。○身長が低いと相手に好意を見せるときに「好き」と思われがちだが、意外とその人の素顔を知ることができるのだ。○美人で色っぽい女性って、自分に自信を持つことだってできるんだよね。○言動が子供っぽさの中にありながら、自分の話をきちっと話せることが大切。○人の性格と同性から見ても好感度はグンと上がる。同性からの嫌われ女の特徴だが、男女間のギャップが最も大きいだろう。同性の女性は同性がモテる理由のひとつとして「草食系」が多いようだ。同性と接する機会は少なくないが、同性から嫌われることの特徴としては、「社交性が高い女性」である。同性

□ 教師データ本文
女の敵は女。職場ではお局様をはじめ女性社員との付き合い、結婚をすれば姑・小姑との付き合い、子供ができればママ友との付き合いに悩む女性たちは多い。同性に嫌われては平和な日常生活を過ごすことができないといっても過言ではない。嫌われないようにするにはどうすればいいのか。まずは周りの独女たちにどんな女性が嫌いかを聞いてみた。○有名人の友達がいるとか、友達の別荘に招待されたとか、自慢ではないことを自慢する。○男性がいると声のトーンが一オクターブ上がる。○若く見えないのに、自分が若く見られると勘違いしている。○何に自信を持っているか分からないけど絶えず上から目線。○人の批判が多い。○男性の前で態度ががらりと変わってぶりっこになる。○いつも自分が一番でなければ気が済まない、女王様気質。○自分勝手でまわりを振り回す。○性格に裏表がある。○普段はぼっとしているのに、男性の前になる

In [ ]:
index = 13
title = valid_data[index][0]
body = valid_data[index][1]
generated_texts = generate_text_from_model(
    title=title, trained_model=trained_model, tokenizer=tokenizer, num_return_sequences=1
)
print('□ タイトル')
print(title)
print()
print('□ 生成本文')
print(generated_texts[0])
print()
print('□ 教師データ本文')
print(body)

□ タイトル
自転車女子、はじめましたvol.09「峠こそ、美女を作る!?」presented byゆるっとcafe

□ 生成本文
こんにちは、独女の皆様。お下劣、毒舌、ご満悦!ドロンジョーヌ恩田です。さて、自転車というさわやか丸出しの魅力に憑りつかれまして、その素晴らしさを広めるべく、鋭意活動中の三十路でございます。こればっかりは、老廃物が溜まってくる頃ではないでしょうか?空気が乾燥して身体が荒れたり、関節がブレたりすると、新陳代謝が悪くなって冷え性になったりします。夏は、暑いアスファルトの中を走るため、汗をかいたまま出歩くのもつらいでしょう。そして、そんな運動不足の毎日を少しでも長く乗り切るために、日焼け止めを塗って肌を露出するなんてことも大いにありそうです。それもそのはず。しかし、夏のレジャーシーズンに向け、愛車との長きに渡って愛用されるのが、富士フィルムのスタジオ「canon」。本格的な自転車テクニックには詳しくないので、いまさら聞けない人も多いと思います。ただ、峠道は大変だし、坂道はとにかくキツく走りたいもの。そこでオススメしたいのが、「峠こそ、美女を作る!」というもの。ひと漕ぎ目から飛び出す、静かな山道をザクザクのゆるふわしさ。水たまりやすい場所なんです。砂っぽかった街中で、突然ぶつかりあったり......。あの赤ちゃんみたいじゃなく、

□ 教師データ本文
今月もこんにちは。独女の皆様。お下劣、毒舌、即解決!ドロンジョーヌ恩田です。さて、常日頃から「坂道好き」を公言しているドロンジョーヌですが、やはり多くの人からすると、「なぜ、わざわざ自転車で坂道を?」と疑問に思われることでしょう。ドロンジョーヌだって坂道がキツいと思わないわけではありません。ただ、せっかく自転車で体を動かすなら、坂道を!と思ってしまいます。坂道を走るおもしろさや楽しさ、素晴らしさは、ほかに替えがたいものですから。登坂は、自分ひとりの体力と、自転車という道具だけで、自然へ挑む時間です。自分を信じ、ペースや呼吸を維持しながら、ただひたすら前へ進みます。体全体が軋むような苦しい時間だからこそ、坂道を上りきったときの達成感は最高です。空と自然に抱かれ、何ともいえない満足感。しかも、そのあとはスリルと緊張と高揚感を味わえる「下り坂」というオマケ付き!日常のストレスなんて、すべて坂の途中に置き